In [ ]:
!pip install requests
import urllib.request

!pip install tsplib95
import tsplib95

In [ ]:
import random  # Libreria para generar numeros y listas aleatorias
import copy  # Permite hacer copias de objetos en python: listas, diccionarios,...

In [ ]:
# Librerias y carga del problema

# http://elib.zib.de/pub/mp-testdata/tsp/tsplib/
# Documentacion :
# https://wwwproxy.iwr.uni-heidelberg.de/groups/comopt/software/TSPLIB95/tsp95.pdf
# https://tsplib95.readthedocs.io/usage.html
# https://tsplib95.readthedocs.io/modules.html#module-tsplib95.models

# Descargamos el fichero de datos(Matriz de distancias)
file = "swiss42.tsp"
urllib.request.urlretrieve(
    "http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz",
    file + ".gz",
)
# !gzip -d swiss42.tsp.gz     #Descomprimir el fichero de datos

# Objeto de tsplib95 para nuestro problema problema
problem = tsplib95.load_problem(file)

# Nodos
Nodos = list(problem.get_nodes())

# Aristas
Aristas = list(problem.get_edges())

# Coordenadas(si estan disponibles en el ficher)
problem.get_display(1)

# Distancia
problem.get_weight(1, 2)

# Matriz de distancias
[
    [problem.get_weight(i, j) for i in list(problem.get_nodes())]
    for j in list(problem.get_nodes())
]

In [ ]:
# Funciones de la Actividad Guiada 3
# Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos):
    solucion = [Nodos[0]]
    for n in Nodos[1:]:
        solucion = solucion + [random.choice(list(set(Nodos) - set(solucion)))]
    return solucion


# Devuelve la distancia entre dos nodos
def distancia(a, b, problem):
    return problem.get_weight(a, b)


# Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
    distancia_total = 0
    for i in range(len(solucion) - 1):
        distancia_total += distancia(solucion[i], solucion[i + 1], problem)
    return distancia_total + distancia(
        solucion[len(solucion) - 1], solucion[0], problem
    )

In [ ]:
distancia_total(
    [
        0,
        32,
        34,
        33,
        20,
        35,
        36,
        31,
        17,
        7,
        1,
        6,
        4,
        3,
        27,
        2,
        28,
        29,
        30,
        38,
        22,
        39,
        21,
        24,
        40,
        23,
        41,
        9,
        8,
        10,
        25,
        11,
        12,
        18,
        26,
        5,
        13,
        19,
        14,
        16,
        15,
        37,
    ],
    problem,
)

In [ ]:
# Genera una poblacion inicial de soluciones de tamaño N
def generar_poblacion(Nodos, N):
    return [crear_solucion(Nodos) for _ in range(N)]


# Evalua la población y devuelve el mejor individuo
def Evaluar_Poblacion(poblacion, problem):
    mejor_solucion = []
    mejor_distancia = 10e100
    for p in poblacion:
        # print("solucion:", p)
        distancia_referencia = distancia_total(p, problem)
        if distancia_referencia < mejor_distancia:
            mejor_solucion = p
            mejor_distancia = distancia_referencia
    return mejor_solucion, mejor_distancia


# Funcion de cruce. Recibe una poblacion(lista de soluciones) y devuelve la población ampliada con los hijos.
# Todos los individuos de la población son selecionados para el cruce(si la población es par)
# Podría aplicarse un proceso previo de selección para elegir los individuos que se desea cruzar.
def Cruzar(poblacion, mutacion, problem):
    # Definimos en una variable la copia de la población para ir eliminando los padres seleccionados
    poblacion_copia = copy.deepcopy(poblacion)

    # Definimos en una variable la copia de la población para ir añadiendo los hijos creados
    poblacion_final = copy.deepcopy(poblacion)

    while len(poblacion_copia) > 1:  # Iteramos mientras haya padres disponibles
        # Seleccionamos dos padres
        padre1, padre2 = random.sample(poblacion_copia, 2)
        poblacion_copia.remove(padre1)
        poblacion_copia.remove(padre2)
        poblacion_final.extend(
            Descendencia_UniformCross([padre1, padre2], problem, mutacion)
        )
    return poblacion_final


def Descendencia_SinglePoint(padres, problem, mutacion):
    pc = random.choice(range(len(padres[0])))
    hijo1 = Factibilizar(padres[0][:pc] + padres[1][pc:], problem)
    hijo2 = Factibilizar(padres[1][:pc] + padres[0][pc:], problem)
    return [hijo1, hijo2, Mutar(hijo1, mutacion), Mutar(hijo2, mutacion)]


def Descendencia_UniformCross(parents: list, problem, mutation: float) -> list:
    c1 = []
    c2 = []

    for i in range(len(parents[0])):
        should_keep_order = random.random() > 0.5

        p1, p2 = parents if should_keep_order else parents[::-1]
        c1.append(p1[i])
        c2.append(p2[i])

    c1 = Factibilizar(c1, problem)
    c2 = Factibilizar(c2, problem)

    return [c1, c2, Mutar(c1, mutation), Mutar(c2, mutation)]


# Para el operador de cruce 1-punto los hijos generados no son soluciones(algunos nodos se repiten y otros no están)
def Factibilizar(solucion, problem):
    Nodos = list(problem.get_nodes())
    nodos_desaparecidos = list(set(Nodos) - set(solucion))
    # Recorremos todos los nodos, cuando haya uno que ya esté en la lista los cambiamos por uno de la lista de nodos_desaparecidos
    for i in range(len(solucion)):
        if solucion[i] in solucion[:i]:
            # print("\tSe repite el",solucion[i] )
            # print("\tSe cambia en la posicion ",i , " " ,solucion[i] , " por ", nodos_desaparecidos[0] )
            solucion[i] = nodos_desaparecidos.pop(
                0
            )  # Cambiamos el nodo y a la vez eliminamos el nodo usado de nodos_desaparecidos
    return solucion


# Funcion de mutación. Se eligen dos nodos y se intercambia. Se podrian añadir otros operaradores
# Se hace mutaciones mutacion% de las veces
def Mutar(solucion: list, mutacion):
    if random.random() < mutacion:
        sel1, sel2 = random.sample(range(len(solucion)), 2)
        aux = solucion.copy()
        aux[sel1], aux[sel2] = aux[sel2], aux[sel1]
        return aux

    return solucion


# Funcion de seleccion de la población. Recibe como parametro una poblacion y
# devuelve una poblacion a la que se ha eliminado individuos poco aptos(fitness alto) y para mantener una poblacion estable de N individuos
# Se tiene en cuenta el porcentaje elitismo pasado como parametro
# Para los individuos que no son de la elite podríamos usar una selección de ruleta(proporcional a su fitness)
def Seleccionar(problem, poblacion, N, elitismo):
    # Se ordena la población según el fitness(tamaño del recorrido) en una lista de elementos [distancia, solucion]
    poblacion_ordenada = sorted(
        [[distancia_total(solucion, problem), solucion] for solucion in poblacion],
        key=lambda x: x[0],
    )

    # Devolvemos elitismo% y el resto se eligen aleatoriamente
    return [x[1] for x in poblacion_ordenada][: int(N * elitismo)] + random.sample(
        [x[1] for x in poblacion_ordenada][int(N * elitismo) :], int(N * (1 - elitismo))
    )

In [ ]:
import numpy as np


# Funcion principal del algoritmo genetico
#######################################################3
def algoritmo_genetico(
    problem=problem, N=100, mutacion=0.15, elitismo=0.1, generaciones=100
):
    """_summary_

    Args:
        problem (_type_, optional): problem. Defaults to problem.
        N (int, optional): amount of agents. Defaults to 100.
        mutacion (float, optional): percentage of introducing random changes in genes. Defaults to 0.15.
        elitismo (float, optional): _description_. Defaults to 0.1.
        generaciones (int, optional): _description_. Defaults to 100.

    Returns:
        _type_: _description_
    """
    # problem = datos del problema
    # N = Tamaño de la población
    # mutacion = probabilidad de una mutación
    # elitismo = porcion de la mejor poblacion a mantener
    # generaciones = nº de generaciones a generar para finalizar

    # Genera la poblacion inicial
    Nodos = list(problem.get_nodes())
    poblacion = generar_poblacion(Nodos, N)

    # Inicializamos valores para la mejor solucion
    (mejor_solucion, mejor_distancia) = Evaluar_Poblacion(poblacion, problem)

    elitismo_l = np.linspace(*elitismo, N)
    mutacion_l = np.linspace(*mutacion[::-1], N)

    # empezamos con un elitismo bajo y vamos incrementado a medida que pasan las generacion
    elitismo = 0
    for n in range(generaciones):
        poblacion = Cruzar(poblacion, mutacion_l[n], problem)

        poblacion = Seleccionar(problem, poblacion, N, elitismo_l[n])

        # Evaluamos la nueva población
        (mejor_solucion, mejor_distancia) = Evaluar_Poblacion(poblacion, problem)

        print(
            "Generacion #",
            n + 1,
            "\nLa mejor solución es:",
            mejor_solucion,
            "\ncon distancia ",
            mejor_distancia,
            "\nelitismo ",
            elitismo_l[n],
            "\nmutacion ",
            mutacion_l[n],
            "\n",
        )

    return mejor_solucion


# 1º intento  :2113
# sol = algoritmo_genetico(problem=problem,N=100,mutacion=.15,elitismo=.1,generaciones=200)

# 2º intento. Aumentamos la poblacion:1654
# sol = algoritmo_genetico(problem=problem,N=500,mutacion=.15,elitismo=.1,generaciones=200)

# 3º intento. Aumentamos las generaciones:2055
# sol = algoritmo_genetico(problem=problem,N=500,mutacion=.15,elitismo=.1,generaciones=250)

# 4º intento. Aumentamos el elitismo: 1728
# sol = algoritmo_genetico(problem=problem,N=500,mutacion=.15,elitismo=.40,generaciones=250)

# 5º intento. Aumentamos la mutacion : 1629 [0, 29, 9, 21, 40, 24, 39, 22, 38, 30, 28, 4, 18, 26, 5, 6, 32, 34, 33, 20, 35, 36, 31, 17, 37, 7, 8, 23, 41, 10, 25, 11, 12, 13, 19, 14, 16, 15, 1, 3, 2, 27]
sol = algoritmo_genetico(
    problem=problem, N=300, mutacion=(0.5, 0.8), elitismo=(0.45, 0.9), generaciones=300
)

In [ ]:
import math
import numpy as np

N = 250
elitismo = (0.4, 0.8)
mutacion = (0.3, 0.8)

elitismo_l = np.linspace(*elitismo, N)
mutacion_l = np.linspace(*mutacion[::-1], N)

for i in range(250):
    print("mutation", mutacion_l[i], "elitismo", elitismo_l[i])

In [ ]:
def SinglePointCross(parents):
    p1, p2 = parents

    # pick a random point to split
    i = random.choice(range(1, len(p1) - 1))
    print("Single point Crossover:", i)

    c1 = p1[:i] + p2[i:]
    c2 = p2[:i] + p1[i:]
    return [c1, c2]


def TwoPointCross(parents):
    p1, p2 = parents

    # pick two random points to split
    i1, i2 = sorted(random.sample(range(1, len(p1) - 1), 2))
    print("Two point Crossover:", i1, i2)

    c1 = p1[:i1] + p2[i1:i2] + p1[i2:]
    c2 = p2[:i1] + p1[i1:i2] + p2[i2:]

    return [c1, c2]


def MultiPointCross(parents):
    p1, p2 = parents

    # pick two random points to split
    points = sorted(
        random.sample(range(1, len(p1) - 1), random.randint(2, len(p1) - 2))
    )
    print("Multi point Crossover:", *points)

    c1 = []
    c2 = []

    start_idx = 0

    for split_idx in points:
        c1 += p1[start_idx:split_idx]
        c2 += p2[start_idx:split_idx]

        start_idx = split_idx
        p1, p2 = p2, p1

    c1 += p1[start_idx:]
    c2 += p2[start_idx:]

    return [c1, c2]


def UniformCross(parents):
    c1 = []
    c2 = []

    selection = []

    for i in range(len(parents[0])):
        should_swap = random.random() > 0.5
        selection.append("p1" if should_swap else "p2")

        p1, p2 = parents if should_swap else parents[::-1]
        c1.append(p1[i])
        c2.append(p2[i])

    print("Uniform Crossover:", *selection)

    return [c1, c2]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# amount of genes
n_genes = 10

p1, p2 = [list(range(n_genes)), list(range(n_genes))[::-1]]

runs = [
    {"name": "Single-point crossover", "fn": SinglePointCross},
    {"name": "Two-point crossover", "fn": TwoPointCross},
    {"name": "Multi-point crossover", "fn": MultiPointCross},
    {"name": "Uniform crossover", "fn": UniformCross},
]

xticklabels = range(1, n_genes + 1)
yticklabels = ["Parent 1", "Parent 2", "Child 1", "Child 2"]

# Create a figure and a set of subplots (1 row, 2 columns)
fig, axs = plt.subplots(len(runs))


for i, run in enumerate(runs):
    c1, c2 = run["fn"]([p1, p2])

    data = [p1, p2, c1, c2]
    print(data)
    sns.heatmap(
        data,
        cmap="coolwarm",
        yticklabels=yticklabels,
        ax=axs[i],
    )
    axs[i].set_title(run["name"])

plt.tight_layout()
plt.show()